In [1]:
## SEED
import os
seed_value = 42
os.environ['PYTHONHASHSEED']=str(seed_value)

## Libraries
import numpy as np
import pandas as pd
import random
import pydicom
from PIL import Image
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg16_preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import preprocess_input as resnet_preprocess_input
from tensorflow.image import resize, grayscale_to_rgb

## For using numpy in tensorflow predictions
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

## Set seed for reproducibility and make a function to keep the seeds constant for loops
def set_seed(seed_value):
    os.environ['PYTHONHASHSEED']=str(seed_value)
    tf.random.set_seed(seed_value)
    np.random.seed(seed_value)
    random.seed(seed_value)

## Set memory growth for GPU
physical_devices = tf.config.experimental.list_physical_devices('GPU')
for physical_device in physical_devices: 
    tf.config.experimental.set_memory_growth(physical_device, True)

## Ignore warnings
# import warnings
# warnings.filterwarnings("ignore")

In [11]:
pretrained = 'vgg16'
img_size = (224, 224)
test_dicoms_path = './data/'
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']
model_path = './models/v10_my_model_xray_udacity.json'

In [36]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
def check_dicom(filename): 
    #print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)       
    if ds.Modality == 'DX' and ds.BodyPartExamined == 'CHEST' and (ds.PatientPosition == 'PA' or ds.PatientPosition == 'AP'):
        img = ds.pixel_array
        return img, ds
    else:
        return None, None
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input

def preprocess_image(img, img_size):

    img = Image.fromarray(img)
    
    proc_img = np.array(img.resize(size=img_size, resample=Image.Resampling.NEAREST))
    
    proc_img = grayscale_to_rgb(tf.reshape(proc_img, (1, *tuple(img_size), 1)))

    return vgg16_preprocess_input(np.array(proc_img))

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path):
    with open(model_path, 'r') as json_file:
        model_json = json_file.read()
    model = tf.keras.models.model_from_json(model_json)
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    prob = model.predict(img, verbose=0)
    if prob[0][0] >= thresh:
        return 1, prob
    else:
        return 0, prob


In [37]:
my_model = load_model(model_path)
thresh = 0.535
dict_img = {'id':[],'filename': [], 'prediction': [], 'probability': [], 'finding': [], 'age':[], 'sex':[], 'position':[], 'modality':[], 'body_part':[]}


for i in test_dicoms:
    set_seed(seed_value)
    full_path = test_dicoms_path + i
    img = np.array([])
    img , ds = check_dicom(full_path)
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img, img_size)
    pred, prob = predict_image(my_model,img_proc,thresh)

    dict_img['id'].append(ds.PatientID)
    dict_img['filename'].append(i)
    dict_img['prediction'].append(pred)
    dict_img['probability'].append(round(prob[0][0], 3))
    dict_img['finding'].append(ds.StudyDescription)
    dict_img['age'].append(int(ds.PatientAge))
    dict_img['sex'].append(ds.PatientSex)
    dict_img['position'].append(ds.PatientPosition)
    dict_img['modality'].append(ds.Modality)
    dict_img['body_part'].append(ds.BodyPartExamined)

df = pd.DataFrame(dict_img)
df

2023-02-02 11:29:04.130774: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-02-02 11:29:04.290878: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-02-02 11:29:04.444450: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


,id,filename,prediction,probability,finding,age,sex,position,modality,body_part
0,2,test1.dcm,0,0.496,No Finding,81,M,PA,DX,CHEST
1,1,test2.dcm,0,0.492,Cardiomegaly,58,M,AP,DX,CHEST
2,61,test3.dcm,0,0.487,Effusion,77,M,AP,DX,CHEST
